In [1]:
!pip install pretrainedmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 1.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 6.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 23.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 12.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 4.6 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 7.0 

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import seaborn as sns
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image
import random

/home/user/Desktop/Challenge/challenge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#catalog_eng= pd.read_csv("/kaggle/input/textphase1/data/catalog_english_taxonomy.tsv",sep="\t")
X_train= pd.read_csv("X_train_update.csv",sep=",")
Y_train= pd.read_csv("Y_train_CVw08PX.csv",sep=",")
X_test=pd.read_csv("X_test_update.csv",sep=",")

# Renommer la première colonne
X_train.rename(columns={X_train.columns[0]: "integerid"}, inplace=True)
Y_train.rename(columns={Y_train.columns[0]: "integerid"}, inplace=True)
X_test.rename(columns={X_test.columns[0]: "integerid"}, inplace=True)

dict_code_to_id = {}
dict_id_to_code={}
list_tags = list(Y_train['prdtypecode'].unique())

print(X_train.shape)
print(Y_train.shape)

for i,tag in enumerate(list_tags):
    dict_code_to_id[tag] = i 
    dict_id_to_code[i]=tag
Y_train['labels']=Y_train['prdtypecode'].map(dict_code_to_id)
train=pd.merge(left=X_train,right=Y_train,
               how='left',on=['integerid'],)
#prod_map=pd.Series(catalog_eng['Top level category'].values,index=catalog_eng['prdtypecode']).to_dict()
#train['product']=train['prdtypecode'].map(prod_map)

def get_img_path(img_id,prd_id,path):
    
    pattern = 'image'+'_'+str(img_id)+'_'+'product'+'_'+str(prd_id)+'.jpg'
    return path + pattern
train_img = train[['imageid','productid','labels']]

train_img['image_path']=train_img.progress_apply(lambda x: get_img_path(x['imageid'],x['productid'],
                                                                path = './images/image_train/'),axis=1)
X_test['image_path']=X_test.progress_apply(lambda x: get_img_path(x['imageid'],x['productid'],
                                                    path='./images/image_test'),axis=1)
train_df, val_df, _, _ = train_test_split(train_img, train_img['labels'],random_state=2020, test_size = 0.1, stratify=train_img['labels'])



(84916, 5)
(84916, 2)


100%|██████████| 84916/84916 [00:00<00:00, 291925.59it/s]
/tmp/ipykernel_8844/2473055396.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_img['image_path']=train_img.progress_apply(lambda x: get_img_path(x['imageid'],x['productid'],
100%|██████████| 13812/13812 [00:00<00:00, 284907.80it/s]


In [10]:
list_labs = list(train_img['labels'].unique())

In [11]:
train_img.isna().sum()

imageid       0
productid     0
labels        0
image_path    0
dtype: int64

## Transfer Learning with PyTorch

In [39]:
!pip install torchsummary

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from torch.utils.data import DataLoader
import os
import copy
from torchsummary import summary
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  2.5.1+cu124
Torchvision Version:  0.20.1+cu124


In [41]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = len(dict_code_to_id)

# Batch size for training (change depending on how much memory you have)
batch_size = 64

# Number of epochs to train for
epochs = 10

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
# feature_extract = True

#### Data Augmentation


The transform RandomResizedCrop crops the input image by a random size(within a scale range of 0.8 to 1.0 of the original size and a random aspect ratio in the default range of 0.75 to 1.33 ). The crop is then resized to 256×256.

RandomRotation rotates the image by an angle randomly chosen between -15 to 15 degrees.

RandomHorizontalFlip randomly flips the image horizontally with a default probability of 50%.

CenterCrop crops an 224×224 image from the center.

ToTensor converts the PIL Image which has values in the range of 0-255 to a floating point Tensor and normalizes them to a range of 0-1, by dividing it by 255.

Normalize takes in a 3 channel Tensor and normalizes each channel by the input mean and standard deviation for the channel. Mean and standard deviation vectors are input as 3 element vectors. Each channel in the tensor is normalized as T = (T – mean)/(standard deviation)

In [42]:
input_size = 224 # for Resnt
# Applying Transforms to the Data

image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.Resize(size=256),
        transforms.CenterCrop(size=input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [43]:
# Load the Data
from torch.utils.data import Dataset, DataLoader, Subset

class ImageDataset(Dataset):
    
    def __init__(self,df,transform=None,mode='train'):
        self.df = df
        self.transform=transform
        self.mode=mode
            
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        
        im_path = self.df.iloc[idx]['image_path']
        img = cv2.imread(im_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img=Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)
        img=img.cuda()
        
        if self.mode=='test':
            return img
        else:
            labels = torch.tensor(self.df.iloc[idx]['labels']).cuda()
            return img, labels

In [44]:
train_dataset=ImageDataset(df=train_df,transform=image_transforms['train'])
val_dataset=ImageDataset(df=val_df,transform=image_transforms['valid'])
test_dataset=ImageDataset(df=X_test,transform=image_transforms['test'],mode='test')


In [45]:
train_data=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
valid_data=DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
test_data=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

### Load the pre-trained model

In [46]:
from torch.nn import functional as F
import torch.nn as nn
import pretrainedmodels
class SEResnext50_32x4d(nn.Module):
    def __init__(self, pretrained='imagenet'):
        super(SEResnext50_32x4d, self).__init__()
        
        self.base_model = pretrainedmodels.__dict__["se_resnext50_32x4d"](pretrained=None)
        if pretrained is not None:
            self.base_model.load_state_dict(
                torch.load("../input/pretrained-model-weights-pytorch/se_resnext50_32x4d-a260b3a4.pth"
                )
            )
        self.l0 = nn.Linear(2048, num_classes)
    
    def forward(self, image):
        batch_size, _, _, _ = image.shape
        
        x = self.base_model.features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        
        out = self.l0(x)

        return out


In [47]:
# Vérifier si CUDA est disponible
if torch.cuda.is_available():
    print("CUDA est disponible")
    print("Nom du GPU :", torch.cuda.get_device_name(0))
    print("Nombre de GPU disponibles :", torch.cuda.device_count())
else:
    print("CUDA n'est pas disponible")


CUDA est disponible
Nom du GPU : NVIDIA GeForce RTX 4090
Nombre de GPU disponibles : 1


When a model is loaded in PyTorch, all its parameters have their ‘requires_grad‘ field set to true by default. That means each and every change to the parameter values will be stored in order to be used in the back propagation graph used for training. This increases memory requirements. So, since most of the parameters in our pre-trained model are already trained for us, we reset the requires_grad field to false.

In [50]:
model = models.resnext50_32x4d(pretrained="imagenet")
model.cuda()

/home/user/Desktop/Challenge/challenge/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/Desktop/Challenge/challenge/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /home/user/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:02<00:00, 43.2MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [52]:
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5          [-1, 128, 56, 56]           8,192
       BatchNorm2d-6          [-1, 128, 56, 56]             256
              ReLU-7          [-1, 128, 56, 56]               0
            Conv2d-8          [-1, 128, 56, 56]           4,608
       BatchNorm2d-9          [-1, 128, 56, 56]             256
             ReLU-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          32,768
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

Next, we define the loss function and the optimizer to be used for training. PyTorch provides many kinds of loss functions. We use the Negative Loss Likelihood function as it can be used for classifying multiple classes. PyTorch also supports multiple optimizers. We use the Adam optimizer. Adam is one the most popular optimizers because it can adapt the learning rate for each parameter individually.

In [53]:
# Define Optimizer and Loss Function
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [54]:
from sklearn.metrics import f1_score
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [56]:

'''
Loop to train and validate
Parameters
    :param model: Model to train and validate
    :param loss_criterion: Loss Criterion to minimize
    :param optimizer: Optimizer for computing gradients
    :param epochs: Number of epochs (default=25)

Returns
    model: Trained Model with best validation accuracy
    history: (dict object): Having training loss, accuracy and validation loss, accuracy
'''
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
start = time.time()
history = []
best_f1 = 0
for epoch in range(epochs):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, epochs))
    print('Training')
    # Set to training mode
    model.train()

    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0

    valid_loss = 0.0
    valid_acc = 0.0
    for i, (inputs, labels) in tqdm(enumerate(train_data)):

        inputs = inputs.to(device)
        labels = labels.to(device)

        # Clean existing gradients
        optimizer.zero_grad()
        
        # Forward pass - compute outputs on input data using the model
        outputs = model(inputs)

        # Compute loss
        loss = loss_criterion(outputs, labels)

        # Backpropagate the gradients
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Compute the total loss for the batch and add it to train_loss
        train_loss += loss.item() 


    # Validation - No gradient tracking needed
    true_labels=[]
    predictions=[]
    with torch.no_grad():

        # Set to evaluation mode
        model.eval()

        # Validation loop
        print('Validation')
        for j, (inputs, labels) in tqdm(enumerate(valid_data)):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            loss = loss_criterion(outputs, labels)

            # Compute the total loss for the batch and add it to valid_loss
            valid_loss += loss.item() 




            # Move logits and labels to CPU ------------------------ Our addition ---------------------------
            logits = outputs.detach().cpu().numpy()
            predicted_labels = np.argmax(logits,axis=-1)
            predictions.extend(predicted_labels)
            labels = labels.to('cpu').numpy()
            true_labels.extend(labels)

            # -----------------------------------------------------------------------------------------------
            # Compute total accuracy in the whole batch and add to valid_acc
            valid_acc += flat_accuracy(logits, labels)

    curr_f1=f1_score(true_labels,predictions,average='macro')
    if curr_f1 > best_f1:
        best_f1=curr_f1
        torch.save(model.state_dict(), 'best_model.pt')

    # Find average training loss and training accuracy
    avg_train_loss = train_loss / len(train_data) 

    # Find average validation loss and validation accuracy
    avg_valid_loss = valid_loss/len(valid_data)  
    avg_valid_acc = valid_acc/len(valid_data) 


    # Report the final accuracy for this validation run.
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Validation Loss: {0:.2f}".format(avg_valid_loss))

    print("Validation F1-Score: {}".format(f1_score(true_labels,predictions,average='macro')))
    history.append([avg_train_loss, avg_valid_loss, avg_valid_acc])

    epoch_end = time.time()

#         print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

    # Save if the model has best accuracy till now
#         torch.save(model, dataset+'_model_'+str(epoch)+'.pt')



Epoch: 1/10
Training


1195it [05:37,  3.54it/s]


Validation


133it [00:26,  5.07it/s]


  Average training loss: 1.95
  Validation Loss: 1.72
Validation F1-Score: 0.4008389023005678
Epoch: 2/10
Training


...

  Average training loss: 1.03
  Validation Loss: 1.67
Validation F1-Score: 0.5009928362575659
Epoch: 10/10
Training


1195it [05:25,  3.67it/s]


Validation


133it [00:25,  5.30it/s]


  Average training loss: 0.97
  Validation Loss: 1.26
Validation F1-Score: 0.5788070640008635


## Prediction for validation data

In [60]:
# Put model in evaluation mode
model = models.resnext50_32x4d(pretrained=None)
model.load_state_dict(torch.load('best_model.pt'))
model.cuda()
model.eval()
# Tracking variables 
predictions = []
softmax_logits=[]
true_labels=[]
# Predict 
# Telling the model not to compute or store gradients, saving memory and 
# speeding up prediction

with torch.no_grad():
    for j, (inputs, labels) in tqdm(enumerate(valid_data)):
        inputs = inputs.to(device)
          # Forward pass, calculate logit predictions
        logits = model(inputs)
        #----- Add softmax---     
        m = torch.nn.Softmax(dim=1)
        output = m(logits)
        #-------#------
        output = output.detach().cpu().numpy()
        # Move logits and labels to CPU
        labels = labels.to('cpu').numpy()
        logits = logits.detach().cpu().numpy()
        predictions.extend(np.argmax(logits,axis=-1))
        softmax_logits.extend(output)
        true_labels.extend(labels)
print(f1_score(predictions,true_labels,average='macro'))
print('Prediction on validation DONE')
softmax_logits=np.array(softmax_logits)

print(softmax_logits.shape)

np.save('Valid_resnext50_32x4d_phase1_softmax_logits.npy',softmax_logits)

/home/user/Desktop/Challenge/challenge/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/Desktop/Challenge/challenge/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_8844/4242041195.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_

0.5788070640008635
Prediction on validation DONE
(8492, 1000)


### Prediction for test data

In [1]:
# # Tracking variables 
# predictions = []
# softmax_logits=[]
# # Predict 
# # Telling the model not to compute or store gradients, saving memory and 
# # speeding up prediction

# with torch.no_grad():
#     for i,inputs in tqdm(enumerate(test_data)):
#         inputs = inputs.to(device)
#           # Forward pass, calculate logit predictions
#         logits = model(inputs)
#         #----- Add softmax---     
#         m = torch.nn.Softmax(dim=1)
#         output = m(logits)
#         #-------#------
#         output = output.detach().cpu().numpy()

#         # Move logits and labels to CPU
#         logits = logits.detach().cpu().numpy()
#         predictions.extend(np.argmax(logits,axis=-1))
#         softmax_logits.extend(output)

# print('Inference DONE')

In [ ]:
# softmax_logits=np.array(softmax_logits)

# print(softmax_logits.shape)

# np.save('Test_resnext50_32x4d_phase1_softmax_logits.npy',softmax_logits)

In [ ]:
#len(predictions)

In [ ]:
# X_test['prediction_model']= predictions
# X_test['Prdtypecode']=X_test['prediction_model'].map(dict_id_to_code)
# X_test['Prdtypecode'].value_counts()


In [ ]:
# X_test=X_test.drop(['prediction_model','Title','Description'],axis=1)

In [ ]:
# X_test.to_csv('y_test_task1_phase1_pred.tsv',sep='\t',index=False)
